In [140]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

In [141]:
import matplotlib.pyplot as plt
from matplotlib import rc

# 한글 폰트 설정 (예: 맑은 고딕)
rc('font', family='Malgun Gothic')

# 음수 기호 설정 (한글 폰트와 함께 깨지지 않도록 처리)
plt.rcParams['axes.unicode_minus'] = False

In [142]:
train=pd.read_csv("./data/train.csv")
test=pd.read_csv('./data/test.csv')

### 1. 시술 시기 코드

In [144]:
train['시술 시기 코드'].value_counts()

시술 시기 코드
TRDQAZ    38969
TRCMWS    38090
TRYBLT    36713
TRVNRY    36173
TRJXFG    36031
TRZKPL    35544
TRXQMD    34831
Name: count, dtype: int64

In [145]:
test['시술 시기 코드'].value_counts()

시술 시기 코드
TRDQAZ    13610
TRCMWS    13392
TRYBLT    12929
TRZKPL    12793
TRJXFG    12764
TRVNRY    12560
TRXQMD    12019
Name: count, dtype: int64

In [146]:
a=pd.crosstab(train['시술 시기 코드'], train['임신 성공 여부'],margins=True)
b=pd.crosstab(train['시술 시기 코드'], train['임신 성공 여부'], normalize='index')
pd.concat([a,b],axis=True)

임신 성공 여부,0,1,All,0,1
시술 시기 코드,,,,,
TRCMWS,28285,9805,38090,0.742583,0.257417
TRDQAZ,29405,9564,38969,0.754574,0.245426
TRJXFG,26437,9594,36031,0.733729,0.266271
TRVNRY,26776,9397,36173,0.740221,0.259779
TRXQMD,25910,8921,34831,0.743878,0.256122
TRYBLT,26834,9879,36713,0.730913,0.269087
TRZKPL,26476,9068,35544,0.744880,0.255120
All,190123,66228,256351,NaN,NaN


각 시술 시기 별 데이터의 개수가 유사하며, 임신 성공 확률 역시 비교적 유사하게 나타남. ->원-핫 인코딩

### 2. 시술 당시 나이

In [149]:
train['시술 당시 나이'].value_counts()

시술 당시 나이
만18-34세    102476
만35-37세     57780
만38-39세     39247
만40-42세     37348
만43-44세     12253
만45-50세      6918
알 수 없음        329
Name: count, dtype: int64

In [150]:
a=pd.crosstab(train['시술 당시 나이'], train['임신 성공 여부'],margins=True)
b=pd.crosstab(train['시술 당시 나이'], train['임신 성공 여부'], normalize='index')
pd.concat([a,b],axis=True)

임신 성공 여부,0,1,All,0,1
시술 당시 나이,,,,,
만18-34세,69415,33061,102476,0.677378,0.322622
만35-37세,41694,16086,57780,0.721599,0.278401
만38-39세,30725,8522,39247,0.782862,0.217138
만40-42세,31395,5953,37348,0.840607,0.159393
만43-44세,10807,1446,12253,0.881988,0.118012
만45-50세,5758,1160,6918,0.832321,0.167679
알 수 없음,329,0,329,1.000000,0.000000
All,190123,66228,256351,NaN,NaN


null값이 존재하진 않으나 대신 '알 수 없음'이라는 값으로 대체 되어 있음. 이 데이터는 임신 성공 확률이 0에 가까우므로, 최빈값이 아닌,
임신 성공률이 제일 낮은 만 43-44세로 대체

In [152]:
train.loc[train['시술 당시 나이'] == '알 수 없음', '시술 당시 나이'] = '만 43-44세'


In [153]:
test['시술 당시 나이'].value_counts()

시술 당시 나이
만18-34세    36178
만35-37세    20211
만38-39세    13611
만40-42세    13266
만43-44세     4272
만45-50세     2413
알 수 없음       116
Name: count, dtype: int64

In [154]:
test.loc[test['시술 당시 나이']=='알 수 없음', '시술 당시 나이']='만 43-44세'

나이는 라벨 인코딩 사용하기

### 3. 임신 시도 또는 마지막 임신 경과 연수

In [157]:
train.drop(columns='임신 시도 또는 마지막 임신 경과 연수', inplace=True)

In [158]:
test.drop(columns='임신 시도 또는 마지막 임신 경과 연수', inplace=True)

존재하는 데이터에 비해 결측치가 훨씬 많으므로 삭제

### 4. 시술 유형

In [161]:
train['시술 유형'].value_counts()

시술 유형
IVF    250060
DI       6291
Name: count, dtype: int64

In [162]:
test['시술 유형'].value_counts()

시술 유형
IVF    87891
DI      2176
Name: count, dtype: int64

In [163]:
a=pd.crosstab(train['시술 유형'], train['임신 성공 여부'],margins=True)
b=pd.crosstab(train['시술 유형'], train['임신 성공 여부'], normalize='index')
pd.concat([a,b],axis=True)

임신 성공 여부,0,1,All,0,1
시술 유형,,,,,
DI,5480,811,6291,0.871086,0.128914
IVF,184643,65417,250060,0.738395,0.261605
All,190123,66228,256351,NaN,NaN


얘는 원-핫

### 5. 특정 시술 유형

In [166]:
train['특정 시술 유형'].value_counts()

특정 시술 유형
ICSI                                   122368
IVF                                     91755
Unknown                                 26939
IUI                                      6100
ICSI:ICSI                                2314
ICSI / BLASTOCYST                        1609
IVF / BLASTOCYST                         1248
IVF:IVF                                  1146
ICSI:IVF                                  873
ICSI / AH                                 769
IVF:ICSI                                  392
IVF / AH                                  319
ICSI:Unknown                              207
IVF:Unknown                               100
ICI                                        96
Generic DI                                 71
IVI                                        23
ICSI / BLASTOCYST:IVF / BLASTOCYST          6
ICSI / BLASTOCYST :IVF / BLASTOCYST         4
FER                                         3
IVF / AH:ICSI / AH                          2
GIFT                     

In [202]:
# ":"가 포함된 경우, 첫 번째 값만 남기기 (float값이 있으면 그대로 두기)
train['특정 시술 유형'] = train['특정 시술 유형'].apply(lambda x: x.split(':')[0] if isinstance(x, str) else x)

# 결과 확인
train['특정 시술 유형'].value_counts()


특정 시술 유형
ICSI                  125762
IVF                    93393
Unknown                26939
IUI                     6100
ICSI / BLASTOCYST       1614
IVF / BLASTOCYST        1248
ICSI / AH                771
IVF / AH                 321
기타                       201
Name: count, dtype: int64

In [197]:
# ":"가 포함된 경우, 첫 번째 값만 남기기 (float값이 있으면 그대로 두기)
test['특정 시술 유형'] = test['특정 시술 유형'].apply(lambda x: x.split(':')[0] if isinstance(x, str) else x)

# 결과 확인
test['특정 시술 유형'].value_counts()

특정 시술 유형
ICSI                  44400
IVF                   32931
Unknown                9216
IUI                    2114
ICSI / BLASTOCYST       584
IVF / BLASTOCYST        389
ICSI / AH               264
IVF / AH                105
기타                       64
Name: count, dtype: int64

In [199]:
train.loc[train['특정 시술 유형'].isin(['ICI', 'Generic DI', 'IVI', 'ICSI / BLASTOCYST','FER','GIFT']), '특정 시술 유형'] = '기타'


In [195]:
test.loc[test['특정 시술 유형'].isin(['ICI', 'Generic DI', 'IVI', 'ICSI / BLASTOCYST']), '특정 시술 유형'] = '기타'


세부 수술 정보 삭제 후 주요 수술 정보만 남김.

100개 이하인 수술은 기타로 너무 많은 변수가 생기는 것 방지 

-> 라벨 인코딩 하기

In [204]:
a=pd.crosstab(train['특정 시술 유형'],train['임신 성공 여부'], margins=True)
b=pd.crosstab(train['특정 시술 유형'], train['임신 성공 여부'],normalize='index')
pd.concat([a,b],axis=True)

임신 성공 여부,0,1,All,0,1
특정 시술 유형,,,,,
ICSI,92128,33634,125762,0.732558,0.267442
ICSI / AH,603,168,771,0.782101,0.217899
ICSI / BLASTOCYST,1038,576,1614,0.643123,0.356877
IUI,5316,784,6100,0.871475,0.128525
IVF,69263,24130,93393,0.741629,0.258371
IVF / AH,262,59,321,0.816199,0.183801
IVF / BLASTOCYST,791,457,1248,0.633814,0.366186
Unknown,20549,6390,26939,0.762797,0.237203
기타,171,30,201,0.850746,0.149254


In [216]:
# '특정 시술 유형' 컬럼이 존재하는지 확인하고, NaN 값을 'Unknown'으로 채우기
train['특정 시술 유형'] = train['특정 시술 유형'].fillna('Unknown')


In [218]:
test['특정 시술 유형'] = test['특정 시술 유형'].fillna('Unknown')


Unknown은 일단 보류

### 6. 배란 자극 여부

In [208]:
train['배란 자극 여부'].value_counts()

배란 자극 여부
1    197720
0     58631
Name: count, dtype: int64

자료형이 int hmm

### 7. 배란 유도 유형

In [212]:
train['배란 유도 유형'].value_counts()

배란 유도 유형
기록되지 않은 시행      194432
알 수 없음           61917
세트로타이드 (억제제)         1
생식선 자극 호르몬           1
Name: count, dtype: int64

In [214]:
test['배란 유도 유형'].value_counts()

배란 유도 유형
기록되지 않은 시행    68599
알 수 없음        21468
Name: count, dtype: int64

In [220]:
a=pd.crosstab(train['배란 유도 유형'],train['임신 성공 여부'], margins=True)
b=pd.crosstab(train['배란 유도 유형'], train['임신 성공 여부'],normalize='index')
pd.concat([a,b],axis=True)

임신 성공 여부,0,1,All,0,1
배란 유도 유형,,,,,
기록되지 않은 시행,142245,52187,194432,0.731593,0.268407
생식선 자극 호르몬,0,1,1,0.000000,1.000000
세트로타이드 (억제제),1,0,1,1.000000,0.000000
알 수 없음,47877,14040,61917,0.773245,0.226755
All,190123,66228,256351,NaN,NaN


In [222]:
train.loc[train['배란 유도 유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)']), '배란 유도 유형'] = '기록되지 않은 시행'

매우 극소수인 값들이기에 기록되지 않은 시행으로 변경

In [229]:
a=pd.crosstab(train['배란 유도 유형'],train['배란 자극 여부'], margins=True)
a

배란 자극 여부,0,1,All
배란 유도 유형,,,
기록되지 않은 시행,0,194434,194434
알 수 없음,58631,3286,61917
All,58631,197720,256351


배란 자극 여부가 1인 사람들이 모두 기록되지 않은 시행으로 적혀 있으며, 변수 개수, 유사성이 매우 높음 -> 배란 유도 유형 변수 삭제

In [232]:
train.drop(columns='배란 유도 유형', inplace=True)

In [234]:
test.drop(columns='배란 유도 유형', inplace=True)

### 8. 단일 배아 이식 여부

In [237]:
train['단일 배아 이식 여부'].value_counts()

단일 배아 이식 여부
0.0    191677
1.0     58383
Name: count, dtype: int64

In [239]:
test['단일 배아 이식 여부'].value_counts()

단일 배아 이식 여부
0.0    67431
1.0    20460
Name: count, dtype: int64

### 9. 착상 전 유전 검사 사용 여부

In [243]:
train.drop(columns='착상 전 유전 검사 사용 여부',inplace=True)
test.drop(columns='착상 전 유전 검사 사용 여부',inplace=True)

### 10. 남성 주 불임 원인

In [246]:
train['남성 주 불임 원인'].value_counts()

남성 주 불임 원인
0    249041
1      7310
Name: count, dtype: int64

In [248]:
test['남성 주 불임 원인'].value_counts()

남성 주 불임 원인
0    87459
1     2608
Name: count, dtype: int64

In [255]:
a=pd.crosstab(train['남성 주 불임 원인'],train['임신 성공 여부'], margins=True)
b=pd.crosstab(train['남성 주 불임 원인'], train['임신 성공 여부'],normalize='index')
pd.concat([a,b],axis=True)

임신 성공 여부,0,1,All,0,1
남성 주 불임 원인,,,,,
0,184421,64620,249041,0.740525,0.259475
1,5702,1608,7310,0.780027,0.219973
All,190123,66228,256351,NaN,NaN


이게 임신 여부와 상관이 있을까?

### 11. 남성 부 불임 원인

In [251]:
train['남성 부 불임 원인'].value_counts()

남성 부 불임 원인
0    252989
1      3362
Name: count, dtype: int64

In [253]:
test['남성 부 불임 원인'].value_counts()

남성 부 불임 원인
0    88908
1     1159
Name: count, dtype: int64

In [258]:
a=pd.crosstab(train['남성 부 불임 원인'],train['임신 성공 여부'], margins=True)
b=pd.crosstab(train['남성 부 불임 원인'], train['임신 성공 여부'],normalize='index')
pd.concat([a,b],axis=True)

임신 성공 여부,0,1,All,0,1
남성 부 불임 원인,,,,,
0,187425,65564,252989,0.740842,0.259158
1,2698,664,3362,0.802499,0.197501
All,190123,66228,256351,NaN,NaN
